In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import requests
import time
from sklearn.ensemble import GradientBoostingClassifier
import joblib
import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [98]:
year = 2019
drive_data = pd.DataFrame(requests.get('https://api.collegefootballdata.com/drives?seasonType=regular&year=' + str(year)).json())

In [99]:
game_data = pd.DataFrame(requests.get('https://api.collegefootballdata.com/games?year=' + str(year) + '&seasonType=regular').json())

In [100]:
game_data['game_id'] = game_data['id']
data = pd.merge(drive_data,game_data,on='game_id')
data['drive_id'] = data['id_x']

In [101]:
play_data = pd.DataFrame()
for i in range(15):
    request_df = requests.get('https://api.collegefootballdata.com/plays?seasonType=regular&year=' + str(year) + '&week=' + str(i+1)).json()
    if(len(request_df) == 0):
        break
    time.sleep(2)
    temp_df = pd.DataFrame(request_df)
    temp_df['week'] = i+1
    play_data = play_data.append(temp_df)

In [102]:
pbp_data = pd.merge(play_data,data[['home_team','drive_id']],how='left',on='drive_id')

In [103]:
pbp_data['coef'] = (pbp_data['home_team'] == pbp_data['defense']).astype(int)
pbp_data['adjusted_yardline'] = 100*(1-pbp_data['coef']) +  (2*pbp_data['coef']-1)*pbp_data['yard_line'] #yard_line is defined by home team in API
pbp_data['margin'] = pbp_data['offense_score'] - pbp_data['defense_score']
pbp_data.loc[pbp_data.down > 4, 'down'] = 4
pbp_data.loc[pbp_data.period > 4, 'period'] = 5

In [104]:
data['drive_point'] = data.drive_result.apply(lambda x: 7 if (x == 'TD' or x == 'PUNT TD' or x == 'RUSHING TD' or x == 'PASSING TD') else (3 if (x == 'FG' or x == 'FG GOOD') else (-2 if x == 'SF' else -7 if ( x ==   'PUNT RETURN TD' or x == 'MISSED FG TD' or x == 'INT TD' or x == 'FUMBLE RETURN TD' or x == 'DOWNS TD' or x == 'INT RETURN TOUCH'  or x == 'FG MISSED TD' or x =='PUNT TD' or x == 'TURNOVER ON DOWNS TD') else 0 )))

In [105]:
data['next_drive_point'] = -data['drive_point'].shift(-1).clip_lower(-2)

/Users/parkerf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: clip_lower(threshold) is deprecated, use clip(lower=threshold) instead
  """Entry point for launching an IPython kernel.


In [106]:
data.loc[data.drive_point == 0, 'drive_point'] = data['next_drive_point']

In [107]:
pbp_data = pbp_data.merge(data[['drive_id','drive_point','drive_result']])

In [108]:
exclude_playtype = ['Kickoff',  'End Period',
        'Kickoff Return (Offense)',
       'Kickoff Return Touchdown', 'End of Half', 'Defensive 2pt Conversion','Uncategorized', 'End of Game', 'Timeout']

game_end_drive = ['END OF HALF', 'END OF GAME', 'Uncategorized','END OF 4TH QUARTER', 'DOWNS TD','POSSESSION (FOR OT DRIVES)']

regression_df = pbp_data[~(pbp_data.play_type.isin(exclude_playtype)) & (pbp_data.adjusted_yardline < 100) & (pbp_data.down > 0) &(pbp_data.distance > 0) & (pbp_data.adjusted_yardline>0) & ~(pbp_data.drive_result.isin(game_end_drive))].dropna()

In [109]:
model = smf.mnlogit('drive_point ~ C(down) + distance + adjusted_yardline + C(down):distance + C(down):adjusted_yardline + period + margin', data=regression_df)
result = model.fit()
result.save('19_model.pk1')

Optimization terminated successfully.
         Current function value: 1.301222
         Iterations 14


In [110]:
result = sm.load('19_model.pk1')

In [111]:
special_team_play_type = ['Kickoff','Punt','Kickoff Return (Offense)', 'Kickoff Return Touchdown','Field Goal Good', 'Field Goal Missed', 'Blocked Field Goal',
                          'Blocked Punt','Punt Return Touchdown','Blocked Punt Touchdown','Missed Field Goal Return','Uncategorized', 'Missed Field Goal Return Touchdown','Defensive 2pt Conversion']
timing_play_type = ['End Period','End of Game','Timeout','End of Half']
turnover_play_type = ['Fumble Recovery (Opponent)','Pass Interception Return','Interception Return Touchdown','Fumble Return Touchdown','Safety','Interception','Pass Interception','Punt',
                     'Field Goal Missed', 'Blocked Field Goal', 'Blocked Punt','Punt Return Touchdown','Blocked Punt Touchdown','Missed Field Goal Return', 'Missed Field Goal Return Touchdown']
regular_play_type = [ 'Rush', 'Sack', 'Pass Reception', 'Passing Touchdown','Pass Incompletion', 'Fumble Recovery (Own)','Rushing Touchdown','Pass Interception','Pass Completion']
time_play = ['End Period','Timeout','End of Half','End of Game']
PAT_miss_type= [ 'PAT MISSED','PAT failed', 'PAT blocked', 'PAT BLOCKED']
off_TD = ['Passing Touchdown','Rushing Touchdown']
def_TD = ['Interception Return Touchdown','Fumble Return Touchdown', 'Missed Field Goal Return Touchdown','Blocked Punt Touchdown','Punt Return Touchdown']

In [112]:
regular_play = pbp_data[~pbp_data.play_type.str.contains('Kickoff') & ~(pbp_data.play_type.isin(time_play)) &(pbp_data.down > 0) & (pbp_data.distance > 0)]

In [113]:
CFB_teams_list = pd.read_csv('https://raw.githubusercontent.com/903124/CFB_EPA_data/master/.ipynb_checkpoints/cfb_teams_list-checkpoint.csv',encoding='utf-8')

In [114]:
CFB_teams_list.full_name.unique()

array(['Abilene Christian', 'Air Force', 'Akron', 'Alabama',
       'Alabama A&M', 'Albany', 'Alcorn State', 'Appalachian State',
       'Arizona', 'Arizona State', 'Arkansas', 'Arkansas State',
       'Arkansas-Pine Bluff', 'Army', 'Auburn', 'Austin Peay', 'BYU',
       'Ball State', 'Baylor', 'Bethune-Cookman', 'Boise State',
       'Boston College', 'Bowling Green', 'Buffalo', 'California',
       'Campbell', 'Central Arkansas', 'Central Connecticut',
       'Central Michigan', 'Charleston Southern', 'Charlotte',
       'Cincinnati', 'Clemson', 'Coastal Carolina', 'Colorado',
       'Colorado State', 'Connecticut', 'Delaware State', 'Drake', 'Duke',
       'Duquesne', 'East Carolina', 'Eastern Illinois',
       'Eastern Kentucky', 'Eastern Michigan', 'Eastern Washington',
       'Elon', 'Florida', 'Florida Atlantic', 'Florida International',
       'Florida State', 'Fordham', 'Fresno State', 'Gardner-Webb',
       'Georgia', 'Georgia Southern', 'Georgia State', 'Georgia Tech',
     

In [115]:
regular_play = pd.merge(regular_play,CFB_teams_list,left_on=['offense'],right_on=['full_name'])
regular_play.rename(columns={'abbreviation':'off_abbr', 'full_name': 'off_full_name'}, inplace=True)
regular_play = pd.merge(regular_play,CFB_teams_list,left_on=['defense'],right_on=['full_name'])
regular_play.rename(columns={'abbreviation':'def_abbr', 'full_name': 'def_full_name'}, inplace=True)

In [116]:
EP_predict = result.predict(regular_play[['down','distance','adjusted_yardline', 'period', 'margin']])

In [117]:
EP = EP_predict[0]* -7 + EP_predict[1] * -3 + EP_predict[2] * -2 + EP_predict[4] * 2 + EP_predict[5] * 3 + EP_predict[6] * 7
regular_play['EP_start'] = EP

In [118]:
regular_play['new_yardline']= 0
regular_play['new_down']= 0
regular_play['new_distance']= 0
regular_play['turnover'] = 0

In [119]:
regular_play = regular_play[~pd.isna(regular_play.play_text) & (regular_play.play_type != 'Interception')]

In [120]:

regular_play.loc[regular_play.play_type.isin(turnover_play_type),'new_down'] = 1
regular_play.loc[regular_play.play_type.isin(turnover_play_type),'new_distance'] = 10

regular_play.loc[regular_play.play_text.str.contains('1ST'), 'new_down'] = 1
regular_play.loc[regular_play.play_text.str.contains('1ST'), 'new_distance'] = 10

regular_play.loc[~regular_play.play_type.isin(turnover_play_type) & ~regular_play.play_text.str.contains('1ST'), 'new_down'] = regular_play.down + 1
regular_play.loc[~regular_play.play_type.isin(turnover_play_type) & ~regular_play.play_text.str.contains('1ST'), 'new_distance'] = regular_play.distance - regular_play.yards_gained

regular_play.loc[regular_play.play_text.str.contains('50 yard line'), 'new_yardline'] = 50

In [121]:
regular_play.loc[regular_play.play_type == 'Fumble Recovery (Opponent)', 'new_yardline'] = 100- (regular_play.yard_line + regular_play.yards_gained) 
regular_play.loc[regular_play.play_type == 'Fumble Recovery (Opponent)', 'new_down'] = 1
regular_play.loc[regular_play.play_type == 'Fumble Recovery (Opponent)', 'new_distance'] = 10

regular_play.loc[regular_play.play_type == 'Sack', 'new_yardline'] = 100- (regular_play.yard_line - regular_play.yards_gained)
regular_play.loc[regular_play.play_type == 'Sack', 'new_down'] = regular_play.down + 1
regular_play.loc[regular_play.play_type == 'Sack', 'new_distance'] = regular_play.distance - regular_play.yards_gained

In [122]:

#Collect end of play yardline information (e.g. Alab 38 = Alabama own 38) from play_text and match the team abbreviation

temp_df = regular_play.iloc[np.char.find(regular_play.play_text.values.astype(str), regular_play.off_abbr.values.astype(str)) >= 0] 
temp_df['split_string'] =  [x[1] for x in list(np.char.split(temp_df.play_text.values.astype(str),sep =temp_df.off_abbr.values.astype(str)))]
regular_play.loc[temp_df[temp_df.play_text.str.contains('\d+', regex=True)].index, 'new_yardline'] = 100-np.array(temp_df[temp_df.play_text.str.contains('\d+', regex=True)].split_string.str.extract(r'(\d+)').astype(float)).ravel()

temp_df = regular_play.iloc[np.char.find(regular_play.play_text.values.astype(str), regular_play.def_abbr.values.astype(str)) >= 0]
temp_df['split_string'] =  [x[1] for x in list(np.char.split(temp_df.play_text.values.astype(str),sep =temp_df.def_abbr.values.astype(str)))]
regular_play.loc[temp_df[temp_df.play_text.str.contains('\d+', regex=True)].index, 'new_yardline'] = np.array(temp_df[temp_df.play_text.str.contains('\d+', regex=True)].split_string.str.extract(r'(\d+)').astype(float)).ravel()

/Users/parkerf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/parkerf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [123]:
regular_play.loc[pd.isna(regular_play.new_yardline),'new_distance'] = regular_play.distance - regular_play.yards_gained 
regular_play.loc[pd.isna(regular_play.new_yardline),'new_yardline'] = regular_play.adjusted_yardline - regular_play.yards_gained

regular_play.loc[regular_play.play_type == 'Pass Incompletion', 'new_yardline'] = regular_play.adjusted_yardline

regular_play.loc[regular_play.play_text.str.contains('touchback'), 'new_yardline'] = 80
regular_play.loc[regular_play.play_text.str.contains('touchback'), 'new_down'] = 1

#Fake data for model prediction, EP will be changed after processing the data

regular_play.loc[regular_play.play_type == 'Safety', 'new_yardline'] = 99 #Dummy yardline for Safety

regular_play.loc[regular_play.play_type.isin(off_TD),'new_down'] = 1 #Dummy new down for Offensive tocuhdown play
regular_play.loc[regular_play.play_type.isin(off_TD),'new_distance']  = 10 #Dummy new yards to go for Offensive tocuhdown play

regular_play.loc[(regular_play.play_type.isin(off_TD) | regular_play.play_type.isin(def_TD)),'new_yardline'] = 99  #Dummy yardline for Offensive tocuhdown play

regular_play.loc[(regular_play.play_type == 'Field Goal Good'), 'new_down'] = 1

regular_play.loc[(regular_play.new_down > 4) & ~(regular_play.play_type.isin(off_TD)),'turnover'] = 1 #Turnover on down
regular_play.loc[(regular_play.new_down > 4) & ~(regular_play.play_type.isin(off_TD)),'new_down'] = 1 
regular_play.loc[(regular_play.new_down > 4) & ~(regular_play.play_type.isin(off_TD)),'new_distance'] = 10
regular_play.loc[(regular_play.new_down > 4) & ~(regular_play.play_type.isin(off_TD)),'new_yardline'] = 100-regular_play.new_yardline


regular_play.loc[((regular_play.new_yardline <= 0) |(regular_play.new_distance <= 0))  & (regular_play.play_type == 'Sack') & (regular_play.play_text.str.contains('return')), 'new_down' ] = 1 #Strip sack
regular_play.loc[((regular_play.new_yardline <= 0) |(regular_play.new_distance <= 0)) & (regular_play.play_type == 'Sack') & (regular_play.play_text.str.contains('return')), 'new_distance' ] = 10 
regular_play.loc[((regular_play.new_yardline <= 0) |(regular_play.new_distance <= 0)) & (regular_play.play_text.str.contains('return') & ~(regular_play.play_type.isin(special_team_play_type))), 'new_yardline' ] = 100-(regular_play.adjusted_yardline - regular_play.yards_gained)
regular_play.loc[ regular_play.play_text.str.contains('return'), 'turnover' ] = 1 
regular_play.loc[regular_play.play_type.isin(turnover_play_type),'turnover'] = 1

regular_play.loc[regular_play.new_distance <= 0, 'new_down'] = 1 #First down not in API
regular_play.loc[regular_play.new_distance <= 0, 'new_distance'] = 10

regular_play.loc[regular_play.play_type == 'Field Goal Good', 'new_yardline'] = 20 #Dummy yardline after success field goal
regular_play.loc[regular_play.play_type == 'Field Goal Missed', 'new_yardline'] = (100-regular_play.adjusted_yardline).clip(upper=80)
regular_play.loc[regular_play.play_type == 'Blocked Field Goal', 'new_yardline'] = (100-regular_play.adjusted_yardline)

regular_play.loc[regular_play.play_type == 'Punt', 'new_yardline'] = (100-regular_play.new_yardline)

In [124]:

regular_play.loc[regular_play.new_yardline <= 0 ,'new_yardline'] = regular_play.adjusted_yardline - regular_play.yards_gained

In [125]:
regular_play.loc[regular_play.play_text.str.contains('TOUCHDOWN'),'new_yardline'] = 99

In [126]:

regular_play = regular_play[(regular_play.new_yardline > 0) & (regular_play.new_yardline < 100) & (regular_play.adjusted_yardline > 0) & (regular_play.adjusted_yardline < 100)]

In [127]:
out_df = pd.DataFrame({'down':regular_play['new_down'],'distance':regular_play['new_distance'],'adjusted_yardline':regular_play['new_yardline'],'margin':regular_play['margin'], 'period':regular_play['period']})

In [128]:
EP_predict = result.predict(out_df[['down','distance','adjusted_yardline', 'margin', 'period']])
EP = EP_predict[0]* -7 + EP_predict[1] * -3 + EP_predict[2] * -2 + EP_predict[4] * 2 + EP_predict[5] * 3 + EP_predict[6] * 7

In [129]:
regular_play['EP_end'] = EP

In [130]:
regular_play.loc[(regular_play.play_type.isin(off_TD) | regular_play.play_type.isin(def_TD) | regular_play.play_text.str.contains('TOUCHDOWN') | regular_play.play_text.str.contains(' TD ')  ),'EP_end'] = 7
regular_play.loc[(regular_play.play_type.isin(PAT_miss_type)),'EP_end'] = 6
regular_play.loc[regular_play.play_type == 'Field Goal Good','EP_end'] = 3

In [131]:
regular_play.loc[(regular_play.play_type.isin(turnover_play_type)| regular_play.turnover == 1),'EP_end'] *= -1

In [132]:
regular_play.loc[regular_play.play_type == 'Safety','EP_end'] = -2

In [133]:
regular_play.loc[(regular_play.play_type == 'Extra Point Missed') | (regular_play.play_type == 'Extra Point Good') |(regular_play.play_type == '2pt Conversion') ,'EP_start'] = 0
regular_play.loc[(regular_play.play_type == 'Extra Point Missed'),'EP_end'] = -1
regular_play.loc[(regular_play.play_type == 'Extra Point Good'),'EP_end'] = 0
regular_play.loc[(regular_play.play_type == '2pt Conversion'),'EP_end'] = 1

In [134]:
regular_play['EPA'] = regular_play['EP_end'] - regular_play['EP_start']

In [135]:
pass_play_type = ['Sack','Pass Incompletion','Pass Interception Return','Pass Reception','Interception Return Touchdown','Passing Touchdown','Pass Completion','Pass Interception']
rush_play_type = ['Fumble Recovery (Opponent)','Fumble Recovery (Own)','Fumble Return Touchdown','Rush','Rushing Touchdown']

In [136]:
regular_play[regular_play.play_type.isin(pass_play_type)]['EPA'].mean()

0.020058410080640847

In [137]:
regular_play[regular_play.play_type.isin(rush_play_type)]['EPA'].mean()

-0.043760673124270164

In [138]:
regular_play.groupby('play_type')['EPA'].mean()

play_type
Blocked Field Goal               -1.993945
Blocked Punt                      0.144578
Blocked Punt Touchdown           -4.215560
Defensive 2pt Conversion        -10.510511
Field Goal Good                   1.227778
Field Goal Missed                -1.714962
Fumble Recovery (Opponent)       -4.682209
Fumble Recovery (Own)            -1.057225
Fumble Return Touchdown          -6.355403
Interception Return Touchdown    -7.013349
Missed Field Goal Return         -5.132510
Pass Incompletion                -1.129003
Pass Interception Return         -3.127192
Pass Reception                    0.826021
Passing Touchdown                 3.517076
Penalty                          -0.766350
Punt                              1.024549
Punt Return Touchdown            -4.682101
Rush                             -0.095576
Rushing Touchdown                 2.482116
Sack                             -1.892629
Safety                           -0.618951
Uncategorized                    -0.244823
N

In [139]:
regular_play['passing_player_name'] = np.nan
regular_play['receiving_player_name'] = np.nan
regular_play['rushing_player_name'] = np.nan
regular_play['pass_rush_player_name_1'] = np.nan
regular_play['pass_rush_player_name_2'] = np.nan
regular_play['force_fumble_player'] = np.nan
regular_play['sacked_player_name'] = np.nan
regular_play['intecept_player_name'] = np.nan
regular_play['deflect_player_name'] = np.nan

In [140]:

pass_play_type = ['slant','screen','deep','middle','sideline','crossing']

In [141]:
regular_play.loc[regular_play.play_text.str.contains(' run for ') ,'rushing_player_name'] = regular_play.play_text.str.split(' run for ').str[0]
regular_play.loc[regular_play.play_text.str.contains(' rush ') ,'rushing_player_name'] = regular_play.play_text.str.split(' rush ').str[0]
regular_play.loc[regular_play.play_text.str.contains(' pass ') ,'passing_player_name'] =  regular_play.play_text.str.split(' pass ').str[0].str.split('(crossing|screen|sideline|middle|deep|slant)').str[0]
regular_play.loc[regular_play.play_text.str.contains(' sacked by ') ,'sacked_player_name'] = regular_play.play_text.str.split(' sacked by ').str[0]
regular_play.loc[regular_play.play_text.str.contains(' sacked by ') ,'pass_rush_player_name_1'] = regular_play.play_text.str.split(' sacked by ').str[1].str.split(' for ').str[0].str.split(' and ').str[0]
regular_play.loc[regular_play.play_text.str.contains(' sacked by ') & regular_play.play_text.str.contains(' and '),'pass_rush_player_name_2'] = regular_play.play_text.str.split(' and ').str[1].str.split(' for ').str[0]
regular_play.loc[regular_play.play_text.str.contains(' pass complete to ') ,'receiving_player_name'] = regular_play.play_text.str.split(' pass complete to ').str[1].str.split(' for ').str[0]
regular_play.loc[regular_play.play_text.str.contains(' pass incomplete to ') ,'receiving_player_name'] = regular_play.play_text.str.split(' pass incomplete to ').str[1].str.split(', broken up').str[0].str.replace(r'\b\.$', '', regex=True).str.strip().str.split(', hurried by ').str[0]
regular_play.loc[regular_play.play_text.str.contains(' forced by ') ,'force_fumble_player'] = regular_play.play_text.str.split(' forced by ').str[1].str.split(', ').str[0]
regular_play.loc[regular_play.play_text.str.contains(' pass intercepted ') & ~regular_play.play_text.str.contains(' for a TD '),'intecept_player_name'] = regular_play.play_text.str.split(' pass intercepted ').str[1].str.split(' return ').str[0]
regular_play.loc[regular_play.play_text.str.contains(' pass intercepted ') & ~regular_play.play_text.str.contains(' for a TD ') & regular_play.play_text.str.contains(' at the '),'intecept_player_name'] = regular_play.intecept_player_name.str.split('by ').str[1].str.split(' at the ').str[0]
regular_play.loc[regular_play.play_text.str.contains(' broken up by '), 'deflect_player_name'] = regular_play.play_text.str.split('broken up by ').str[1].str.split('.')[0]
try:
    regular_play.loc[regular_play.play_text.str.contains(' pass intercepted for a TD ') ,'intecept_play_name'] = regular_play.play_text.str.split(' pass intercepted for a TD ').str[1].str.split(' return ').str[0]
except  AttributeError:
    pass

In [142]:
regular_play.to_csv('CFB_regular_play_19.csv')